<a href="https://www.kaggle.com/code/chingiztursunbayev/airline-customer-satisfaction?scriptVersionId=176763049" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviakompaniya/sample_submission.csv
/kaggle/input/aviakompaniya/train_dataset.csv
/kaggle/input/aviakompaniya/test_dataset.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [3]:
train_data = pd.read_csv('/kaggle/input/aviakompaniya/train_dataset.csv')
test_data = pd.read_csv('/kaggle/input/aviakompaniya/test_dataset.csv')

In [4]:
train_data.shape, test_data.shape

((10000, 24), (4000, 23))

In [5]:
test_data.isna().sum()

id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             19
dtype: int64

In [6]:
train_data.isna().sum()

id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             28
satisfaction                          0
dtype: int64

In [7]:
train_data['Arrival Delay in Minutes'] = train_data['Arrival Delay in Minutes'].fillna(28)

test_data['Arrival Delay in Minutes'] = test_data['Arrival Delay in Minutes'].fillna(19)

In [8]:
train_data = pd.get_dummies(train_data, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'], drop_first=True)

In [9]:
train_data.head(10)

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,...,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus
0,1,33,571,2,3,2,4,4,2,4,...,3,4,10,3.0,0,True,True,False,True,False
1,2,49,1431,4,1,4,4,3,5,4,...,5,3,0,0.0,1,False,False,False,False,False
2,3,43,867,1,4,4,4,4,4,3,...,1,2,0,18.0,0,False,False,False,True,False
3,4,27,1550,3,3,3,3,2,2,2,...,4,2,0,0.0,1,False,False,False,False,False
4,5,11,526,3,4,3,2,4,3,4,...,5,4,0,10.0,0,True,False,True,True,False
5,6,30,1034,2,4,2,2,5,2,5,...,4,5,0,0.0,0,True,False,True,False,True
6,7,38,826,5,1,1,1,5,5,5,...,4,5,2,5.0,1,True,False,False,True,False
7,8,39,162,3,4,4,4,3,3,3,...,3,3,0,0.0,1,False,False,False,False,True
8,9,43,541,2,2,2,5,3,2,3,...,4,3,0,0.0,0,True,True,False,False,False
9,10,24,1065,4,4,4,3,2,4,3,...,4,2,10,17.0,0,False,True,False,True,False


In [10]:
X_train = train_data.drop(columns=['id', 'satisfaction'])
y_train = train_data['satisfaction']
X_test = test_data.drop(columns=['id'])

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
X_train_scaled

array([[-0.42793906, -0.64423433, -0.57254957, ..., -0.64236197,
         1.16292204, -0.27456229],
       [ 0.64608723,  0.19706293,  0.89514992, ..., -0.64236197,
        -0.85990287, -0.27456229],
       [ 0.24332737, -0.35467155, -1.30639932, ..., -0.64236197,
         1.16292204, -0.27456229],
       ...,
       [ 0.4447073 ,  0.2058672 ,  0.16130017, ..., -0.64236197,
        -0.85990287, -0.27456229],
       [ 0.10907408, -1.11477384,  0.89514992, ..., -0.64236197,
        -0.85990287, -0.27456229],
       [ 0.10907408,  0.50619075,  0.16130017, ..., -0.64236197,
        -0.85990287, -0.27456229]])

In [13]:
model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [14]:
predictions = model.predict(X_test_scaled)
predictions

array([0, 1, 1, ..., 0, 1, 0])

In [15]:
type(predictions)

numpy.ndarray

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')

mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()

print("Mean Cross-Validation Accuracy:", mean_cv_score)
print("Standard Deviation of Cross-Validation Accuracy:", std_cv_score)

Mean Cross-Validation Accuracy: 0.9479
Standard Deviation of Cross-Validation Accuracy: 0.0023748684174075742


In [17]:
results = pd.DataFrame({'id': test_data['id'], 'satisfaction': predictions})
results.to_csv('predictions.csv', index=False)

In [18]:
results

,id,satisfaction
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
3995,3996,1
3996,3997,0
3997,3998,0
3998,3999,1
